In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [3]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [36]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ACN', 'ADBE', 'ADYEY', 'AMD', 'AMGN',
    'AMZN', 'ANET', 'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD',
    'DDOG', 'DGX', 'DIS', 'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY',
    'FLGT', 'FTNT', 'GH', 'GMED', 'GOOG', 'IDXX', 'INTG', 'ISRG',
    'JD', 'LRCX', 'LSPD', 'LULU', 'MA', 'MASI', 'MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NET', 'NOW', 'NVCR', 'NVDA', 'ODFL', 'OKTA',
    'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS', 'PTON', 'PYPL', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'RIVN', 'RMD', 'ROKU', 'RVLV', 'SAM',
    'SBUX', 'SE', 'SHOP', 'SNOW', 'TASK', 'TDOC', 'TEAM', 'TMUS', 
    'TREX', 'TSCO', 'TSLA', 'TTD', 'TTWO', 'TWLO', 'TWTR', 'TYL',
    'UPST', 'VEEV', 'VLD', 'VRTX', 'XPEV', 'ZEN', 'ZM']
LINGERERS = [
    'ABST', 'AMEH', 'AMRK', 'APT', 'BRTX', 'BTTR', 'BYRN', 'CENN',
    'CGAU', 'CHEF', 'CIZN', 'COOP', 'CTGO', 'CVE', 'DQ', 'DTST',
    'DYAI', 'EP', 'ERIE', 'ESOA', 'FCUV', 'FKWL', 'FSI', 'GGE', 'GME',
    'HBM', 'HMY', 'HNRG', 'IDR', 'INTZ',  'KREF', 'MCVT', 'MICS',
    'MNMD', 'MOBQ', 'NAN', 'OMQS', 'PACB', 'PETV', 'POWW', 'PRPH',
    'RVP', 'RVYL', 'SOBR', 'VIRT', 'WAVD', 'WKHS', 'YORW', 'ZEST',
    'ZYXI']
BEST_WEIGHTED = [
    'AGI', 'CEF', 'CFFN', 'CZFS', 'DGX', 'FCN', 'FNV', 'GCBC', 'GUT',
    'IEI', 'IIM', 'KSM', 'LFMD', 'MIN', 'MIY', 'MMU', 'MTA', 'NAZ',
    'NEN', 'NPV']


SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))
len(SYMBOLS)

157

In [37]:
!ls $DL

Portfolio_Positions_Dec-30-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_12_30.xls
clean.csv


In [38]:
#!head -5 $DL/Positions.csv

In [39]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [40]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [41]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ABST
ACN


### E*Trade

In [42]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [43]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
003CVR016,--
AAPL,3638.04
ABNB,171
ACN,2401.56
ADBE,3028.7699999999995


In [44]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
003CVR016,--
AAPL,3638
ABNB,171
ACN,2402
ADBE,3029


In [45]:
#et_df.drop(index=['98986CONT'], inplace=True)
et_df.head()

,ET
003CVR016,--
AAPL,3638
ABNB,171
ACN,2402
ADBE,3029


In [46]:
#et_df.drop('98936J101', inplace=True)

In [49]:
et_df.loc[et_df.ET == '--']
#et_df.loc['ABMD', 'ET'] = 380.11
#et_df.drop('003CVR016', inplace=True)

,ET


In [50]:
check_symbols(et_df)

All clear


In [51]:
#et_df = et_df[1:]

In [52]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,3638
ABMD,0
ABNB,171
ABST,0
ACN,2402


### Fidelity

In [53]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Dec-30-2022.csv'

In [54]:
!ls $DL

Portfolio_Positions_Dec-30-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_12_30.xls
clean.csv


In [55]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-05-", "-06-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
0,218520300,Rollover IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,1640.47,$1.00,$0.00,$1640.47,NaN,NaN,NaN,NaN,5.07%,NaN,NaN,Cash
1,218520300,Rollover IRA,003CVR016,ABIOMED INC CONTRA,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00%,--,--,Cash
2,218520300,Rollover IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.00,$13.80,-$0.269,$179.40,-$3.50,-1.92%,-$35.29,-16.44%,0.55%,$214.69,$16.51,Cash
3,218520300,Rollover IRA,AMD,ADVANCED MICRO DEVICES INC,18.00,$64.77,-$0.05,$1165.86,-$0.90,-0.08%,-$181.05,-13.45%,3.60%,$1346.91,$74.83,Cash
4,218520300,Rollover IRA,BROS,DUTCH BROS INC CL A,1.00,$28.19,+$0.10,$28.19,+$0.10,+0.35%,-$11.43,-28.85%,0.09%,$39.62,$39.62,Cash


In [56]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
2,Rollover IRA,ADYEY,179
3,Rollover IRA,AMD,1166
4,Rollover IRA,BROS,28
5,Rollover IRA,BYND,49
6,Rollover IRA,CIZN,68


In [57]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['Rollover IRA'].head()

,Rollover IRA
Symbol,
ADYEY,179
AMD,1166
BROS,28
BYND,49
CIZN,68


In [58]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

Rollover IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [59]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA
AAPL,3638,0,520,0
ABMD,0,0,0,0
ABNB,171,0,86,0
ABST,0,0,0,0
ACN,2402,0,1601,0


### TD Ameritrade

In [60]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [61]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

#path = f'{DL}/Positions_damiansp_Stocks_2022_11_04.xls'

In [62]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1].replace(',', ''))
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [63]:
td = parse_file(path)
td.head()

,TD
ABST,41.76
AGI,80.88
AMEH,118.36
AMRK,138.92
APT,140.70


In [64]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,3638,0,520,0,0
ABMD,0,0,0,0,0
ABNB,171,0,86,0,0
ABST,0,0,0,0,42
ACN,2402,0,1601,0,0


In [65]:
out.to_csv(f'{DL}/clean.csv')

In [66]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*